In [1]:
#decrease number of nodes

import os
from math import floor

import Bio.SeqIO as SeqIO
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import load_model

In [2]:
class BatchGenerator(keras.utils.Sequence):
    """Label, batch, and pad protein sequence data.

    Only complete batches are returned, so a single epoch may not train on every example."""
    def __init__(self, records, batch_size, sym_codes, shuffle=True):
        self.records = records
        self.indices = np.arange(len(self.records))
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.ctable = {}  # Category table
        for i, sym_code in enumerate(sym_codes):
            self.ctable[sym_code] = i
        self.on_epoch_end()

    def __len__(self):
        """Return number of batches."""
        return floor(len(self.records) / self.batch_size)

    def __getitem__(self, index):
        """Generate one batch of data."""
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        records = [self.records[i] for i in indices]
        max_len = max([len(record[0]) for record in records])
        x = np.zeros((self.batch_size, max_len))
        y = np.zeros((self.batch_size, max_len))
        for i, (syms, labels) in enumerate(records):
            x[i, :len(syms)] = [self.ctable.get(sym, 0) for sym in syms]
            y[i, :len(syms)] = [int(label) for label in labels]

        x = keras.utils.to_categorical(x, num_classes=len(self.ctable))
        y = keras.utils.to_categorical(y, num_classes=2)
        for i, (syms, _) in enumerate(records):
            x[i, len(syms):, :] = 0
            y[i, len(syms):, :] = 0
        return x, y

    def on_epoch_end(self):
        """Shuffles data after each epoch."""
        if self.shuffle:
            np.random.shuffle(self.indices)

def load_data(seqs_path, labels_path):
    # Load files
    seqs = {}
    for record in SeqIO.parse(seqs_path, 'fasta'):
        accession = record.description.split('|')[0]
        seq = str(record.seq)
        seqs[accession] = seq
    labels = {}
    for record in SeqIO.parse(labels_path, 'fasta'):
        accession = record.description.split('|')[0]
        label = str(record.seq)
        labels[accession] = label

    # Bundle seqs and labels into single object
    records = []
    for accession, seq in seqs.items():
        records.append((seq, labels[accession]))

    return records

def decode(x, sym_codes):
    """Decodes a vector of indices to their amino acid symbols."""
    ctable, i = {0: 'X'}, 1
    for sym_code in sym_codes:
        ctable[i] = sym_code
        i += 1
    records = []
    for indices in x:
        syms = [ctable[index] for index in indices]
        records.append(''.join(syms))
    return records

class MaskedConv1D(keras.layers.Conv1D):
    def __init__(self, filters, kernel_size, **kwargs):
        super().__init__(filters, kernel_size, **kwargs)
        self.supports_masking = True

In [3]:
# Parameters
sym_codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
             'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

In [4]:
# Load data
train_records = load_data('../../mobidb_validation/split_data/out/train_as_fasta.fasta', '../../mobidb_validation/split_data/out/train_labels_as_fasta.fasta')
validation_records = load_data('../../mobidb_validation/split_data/out/val_as_fasta.fasta', '../../mobidb_validation/split_data/out/val_labels_as_fasta.fasta')
test_records = load_data('../../mobidb_validation/split_data/out/test_as_fasta.fasta', '../../mobidb_validation/split_data/out/test_labels_as_fasta.fasta')

In [5]:
# Batch data
train_batches = BatchGenerator(train_records, 32, sym_codes)
validation_batches = BatchGenerator(validation_records, 1, sym_codes)
test_batches = BatchGenerator(test_records, 1, sym_codes)

In [6]:
# Build model
inputs = keras.layers.Input(shape=(None, 20), name='input1')
x = layers.Masking(mask_value=0, name='mask1')(inputs)
x = MaskedConv1D(64, 20, padding='same', activation='relu', name='conv1d1')(x)
x = MaskedConv1D(64, 20, padding='same', activation='relu', name='conv1d2')(x)
outputs = layers.Dense(2, activation='softmax', name='dense1')(x)

model = keras.Model(inputs=inputs, outputs=outputs, name="jason_cnn_7")
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
model.summary()

Model: "jason_cnn_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input1 (InputLayer)          [(None, None, 20)]        0         
_________________________________________________________________
mask1 (Masking)              (None, None, 20)          0         
_________________________________________________________________
conv1d1 (MaskedConv1D)       (None, None, 64)          25664     
_________________________________________________________________
conv1d2 (MaskedConv1D)       (None, None, 64)          81984     
_________________________________________________________________
dense1 (Dense)               (None, None, 2)           130       
Total params: 107,778
Trainable params: 107,778
Non-trainable params: 0
_________________________________________________________________


In [7]:
# Train model
# Epochs are written explicitly in a training loop since Keras
# does not support generators for calculating validation metrics
histories = []
validation_metrics = []
for i in range(50):
    # Fit
    print(f'EPOCH {i}')
    history = model.fit(train_batches, epochs=1)
    histories.append(history)

    # Evaluate
    total = 0
    l_count = 0
    for batch in validation_batches:
        true_labels = batch[1]
        pred_labels = model.predict(batch[0])
        for true_label, pred_label in zip(true_labels, pred_labels):
            idxs = true_label.sum(axis=1).astype(bool)
            total += idxs.sum()
            l_count += (np.argmax(true_label[idxs], axis=1) == np.argmax(pred_label[idxs], axis=1)).sum()
    validation_metrics.append({'total': total, 'l_count': l_count})

    print('LABEL ACCURACY:', l_count / total)
    print()

EPOCH 0
543/543 [==============================] - 275s 507ms/step - loss: 0.0569 - binary_accuracy: 0.9373
LABEL ACCURACY: 0.9307718577280212

EPOCH 1
543/543 [==============================] - 426s 785ms/step - loss: 0.0552 - binary_accuracy: 0.9385
LABEL ACCURACY: 0.9307718577280212

EPOCH 2
543/543 [==============================] - 265s 488ms/step - loss: 0.0552 - binary_accuracy: 0.9384
LABEL ACCURACY: 0.9307718577280212

EPOCH 3
543/543 [==============================] - 274s 505ms/step - loss: 0.0537 - binary_accuracy: 0.9385
LABEL ACCURACY: 0.9307670881101057

EPOCH 4
543/543 [==============================] - 431s 793ms/step - loss: 0.0537 - binary_accuracy: 0.9385
LABEL ACCURACY: 0.9307670881101057

EPOCH 5
543/543 [==============================] - 262s 483ms/step - loss: 0.0535 - binary_accuracy: 0.9386
LABEL ACCURACY: 0.9306421241207209

EPOCH 6
543/543 [==============================] - 293s 539ms/step - loss: 0.0521 - binary_accuracy: 0.9386
LABEL ACCURACY: 0.9306430780

KeyboardInterrupt: 

In [ ]:
# Save model
if not os.path.exists('out/'):
    os.mkdir('out/')

model.save('out/jason_cnn_7.h5')

In [ ]:
model = load_model("out/jason_cnn_7.h5", custom_objects={"MaskedConv1D": MaskedConv1D})

In [ ]:
# Evaluate
total = 0
l_count = 0
for batch in validation_batches:
    true_labels = batch[1]
    pred_labels = model.predict(batch[0])
    for true_label, pred_label in zip(true_labels, pred_labels):
        idxs = true_label.sum(axis=1).astype(bool)
        total += idxs.sum()
        l_count += (np.argmax(true_label[idxs], axis=1) == np.argmax(pred_label[idxs], axis=1)).sum()

print('LABEL ACCURACY:', l_count / total)
print()